In [1]:
import pandas as pd
import os
import sys

utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

from MetadataStats import MetadataStats

In [2]:
fileNames = {
    2:'Septima_cohorte_params.xlsx',
    3:'3ªCohorte_V2_params.xlsx',
    4:'4ªCohorte_V2_params.xlsx',
    5:'5ªCohorte_V2_params.xlsx'
}

In [39]:
mdata = {}

for i in fileNames:
    mdata[i] = pd.read_excel(f'../OriginalFiles/{fileNames[i]}')
    mdata[i]['Cohort'] = i

In [40]:
mdata = pd.concat(mdata.values())

In [41]:
mdata['Caso/control'] = [0 if str(i)[0]=='C' else 1 for i in mdata['CODIGO PROTEO']]
mdata['Smoke_dummy'] = [0 if i in ['No Fumador', 'ExFumador'] else 1 for i in mdata['Tabaco']]

In [42]:
# Add statin data
statin = pd.read_csv(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Alessia\1_DifferentialCorrelation\data\Statin\Metadata_Statin_V2.csv")

In [43]:
mdata = pd.merge(
    statin,#[['Seqn', 'Statin']],
    mdata,
    how='inner', on='Seqn'
)

In [44]:
mdata = pd.merge(
    mdata,
    pd.read_excel('../OriginalFiles/RBR_Seqn2MetaboCode.xlsx'),
    how='inner', on='Seqn'
)
mdata['MetaboCode'] = [str(i) for i in mdata['MetaboCode']]

In [46]:
s2b = pd.read_excel(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA_V2\OriginalFiles\RBR_SamPheno.xlsx")
s2b['sample'] = [str(i.split('_')[1]) for i in s2b['sample']]
mdata = pd.merge(
    mdata,
    s2b,
    how='left',
    left_on='MetaboCode', right_on='sample'
)

In [47]:
mdata.to_csv('main_metadata.tsv', index=False, sep='\t')

In [15]:
mdata

,Seqn,Group,Statin,CODIGO PROTEO,Comparativa,Individuo,Grupo,Plaque_thickness,Calcium_Score,deqage,...,Cystatin C,VCAM1,Plaque_burden,Diastolic Blood Pressure,Framingham 10y,Framingham 30y,Cohort,Caso/control,Smoke_dummy,MetaboCode
0,6,D,0,814,TMT14_127_C,Caso,2.0,20.12,7.39,46,...,0.71,574.400024,313.254639,72,0.058267,0.259810,2,1,0,814
1,8,C,0,C3114,TMT15_129_C,Control,2.0,3.39,0.00,50,...,0.87,681.900024,117.201675,78,0.121574,0.408444,3,0,1,C3114
2,9,C,0,C3313,TMT1_127_N,Control,4.0,0.00,0.00,50,...,0.87,631.500000,0.000000,64,0.041456,0.157476,3,0,0,C3313
3,11,D,0,3127,TMT8_128_C,NaN,NaN,3.20,0.00,41,...,0.65,457.100006,94.076637,75,0.038744,0.108659,5,1,0,3127
4,16,D,0,3125,TMT3_130_N,NaN,NaN,8.94,0.00,49,...,1.05,445.799988,32.232815,68,0.062823,0.260285,5,1,1,3125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,3804,D,0,1003,TMT8_130_N,Caso,1.0,22.34,384.04,54,...,0.80,735.099976,518.492493,79,0.117965,0.302398,2,1,1,1003
436,3844,C,0,C3315,TMT3_127_N,NaN,NaN,0.00,0.00,44,...,0.64,453.000000,0.000000,69,0.027761,0.105787,4,0,0,C3315
437,3860,D,1,3310,TMT2_129_C,Case,4.0,6.95,45.60,51,...,1.00,517.900024,65.390976,67,0.064440,0.254976,3,1,1,3310
438,3861,C,0,C3210,TMT10_129_C,Control,3.0,3.08,0.00,50,...,0.90,761.099976,43.028496,76,0.088912,0.261047,3,0,0,C3210


In [27]:
filePlot = 'MetadataStats.html'
if os.path.exists(filePlot): os.remove(filePlot)

mds = MetadataStats(mdata, file=filePlot)#, groupCol='Caso/control', C=0, D=1)

qualCols = ['Smoke_dummy', 'Statin']
quanCols = [
    'Plaque_thickness',
    'Calcium_Score',
    'Plaque_burden',
    'Total_Cholesterol',
    'HDL',
    'LDL',
    'Ox-LDL',
    'Lipoprotein a',
    'CRP',
    'Framingham 10y',
    'Framingham 30y',
    'deqage',
    'Systolic Blood Pressure',
    'Diastolic Blood Pressure'
]

mds.plotQuanCols(quanCols[:7])
mds.plotQuanCols(quanCols[7:])
mds.plotQualCols(qualCols)

Mann-Whitney U Test

Plaque_thickness
Statistic = 43503.5 | p-value = 5.668150521472547e-48

Calcium_Score
Statistic = 45676.5 | p-value = 3.0974638693276073e-65

Plaque_burden
Statistic = 42426.0 | p-value = 9.323275498591012e-44

Total_Cholesterol
Statistic = 26623.5 | p-value = 0.06919266543942489

HDL
Statistic = 21891.5 | p-value = 0.08346105309487174

LDL
Statistic = 26320.5 | p-value = 0.11184180900587068

Ox-LDL
Statistic = 28523.0 | p-value = 0.001189392949506048

Mann-Whitney U Test

Lipoprotein a
Statistic = 25917.0 | p-value = 0.1978832281152959

CRP
Statistic = 23945.0 | p-value = 0.8482259424224898

Framingham 10y
Statistic = 31417.0 | p-value = 6.254457166206111e-08

Framingham 30y
Statistic = 31704.0 | p-value = 1.8385930922640446e-08

deqage
Statistic = 27780.0 | p-value = 0.007083975664927042

Systolic Blood Pressure
Statistic = 28762.5 | p-value = 0.0006202453454845802

Diastolic Blood Pressure
Statistic = 27313.5 | p-value = 0.019477858472438707

Chi-Square Homogene